In [1]:
import pandas as pd
import numpy as np
import gc

In [4]:
dtype = {
    'ip': np.int32,
    'app': np.int16,
    'device': np.int16,
    'os': np.int16,
    'channel': np.int16,
    'click_time': str,
    #'attributed_time': str,
    'is_attributed': np.uint8,
    'click_id': str
}

In [5]:
%%time
train = pd.read_feather('Data/tr_drop_duplicate.feather',nthreads=4)
test = pd.read_csv('Data/test.csv.zip',dtype=dtype,low_memory=True,parse_dates=['click_time'])

CPU times: user 33.6 s, sys: 10.7 s, total: 44.3 s
Wall time: 55.5 s


In [6]:
test.head()

,click_id,ip,app,device,os,channel,click_time
0,0,5744,9,1,3,107,2017-11-10 04:00:00
1,1,119901,9,1,3,466,2017-11-10 04:00:00
2,2,72287,21,1,19,128,2017-11-10 04:00:00
3,3,78477,15,1,13,111,2017-11-10 04:00:00
4,4,123080,12,1,13,328,2017-11-10 04:00:00


In [7]:
data = pd.concat([train,test],ignore_index=True)
data.head()

,app,attributed_time,channel,click_id,click_time,device,index,ip,is_attributed,os
0,3,NaT,379,NaN,2017-11-06 14:32:21,1,0.0,83230,0.0,13
1,3,NaT,379,NaN,2017-11-06 14:33:34,1,1.0,17357,0.0,19
2,3,NaT,379,NaN,2017-11-06 14:34:12,1,2.0,35810,0.0,13
3,14,NaT,478,NaN,2017-11-06 14:34:52,1,3.0,45745,0.0,13
4,3,NaT,379,NaN,2017-11-06 14:35:08,1,4.0,161007,0.0,13


In [8]:
data["day"] = data['click_time'].dt.day.astype('uint8')
data["hour"] = data['click_time'].dt.hour.astype('uint8')

data['click_time'] = (data['click_time'].astype(np.int64) // 10 ** 9).astype(np.int32)
data['cum_hour'] = (data.day - 6)*24 + data.hour

varcols = ['ip', 'app', 'device', 'os', 'channel',]
for v in varcols:
    data[v] = data[v].astype('category')
    #test[v]=test[v].astype('category')
data = data.drop(columns = ['attributed_time'])
del test, train, varcols
gc.collect()

518

In [9]:
print('ip:{} app:{} device:{} os:{} channel:{}'.
      format(data.ip.nunique(),data.app.nunique(),data.device.nunique(),data.os.nunique(),data.channel.nunique()))

ip:333168 app:730 device:3799 os:856 channel:202
